In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
class MarketPredictor:
    def __init__(self, file_path):
        # Cargar datos directamente desde la ruta de Drive
        df = pd.read_csv(file_path)
        
        # Procesar datos
        df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        
        # Seleccionar características para la predicción
        self.features = ['Open', 'High', 'Low', 'Close']
        
        # Filtrar datos por intervalos de 10 minutos
        df['10min_group'] = df['Timestamp'].dt.floor('10T')
        grouped = df.groupby('10min_group')[self.features].agg({
            'Open': 'first', 
            'High': 'max', 
            'Low': 'min', 
            'Close': 'last'
        }).reset_index()
        
        data = grouped[self.features].values
        
        # Normalizar los datos
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.scaled_data = self.scaler.fit_transform(data)
        
        # Parámetros del modelo
        self.time_steps = 12  # 2 horas de datos previos (12 intervalos de 10 min)
        self.model = None
        self.last_timestamp = grouped['10min_group'].max()
        self.grouped_data = grouped
    
    def create_sequences(self, data):
        X, y = [], []
        for i in range(len(data) - self.time_steps):
            X.append(data[i:i+self.time_steps])
            y.append(data[i+self.time_steps])
        return np.array(X), np.array(y)
    
    def build_lstm_model(self, input_shape, units=50):
        model = Sequential([
            LSTM(units, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(units//2, return_sequences=False),
            Dropout(0.2),
            Dense(units//2, activation='relu'),
            Dense(4)  # Predecir 4 características: Open, High, Low, Close
        ])
        
        model.compile(optimizer=Adam(learning_rate=0.001), 
                      loss='mean_squared_error')
        
        return model
    
    def train_model(self):
        # Crear secuencias
        X, y = self.create_sequences(self.scaled_data)
        
        # Dividir datos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        
        # Construir y entrenar modelo
        self.model = self.build_lstm_model(X_train.shape[1:])
        history = self.model.fit(
            X_train, y_train, 
            epochs=50,  
            batch_size=32, 
            validation_data=(X_val, y_val),
            verbose=1
        )
        
        return history
    
    def predict_next_day(self):
        if self.model is None:
            raise ValueError("Debes entrenar el modelo primero usando train_model()")
        
        # Tomar los últimos 12 pasos (2 horas) como base para la predicción
        last_sequence = self.scaled_data[-self.time_steps:]
        
        # Almacenar predicciones
        predictions = []
        prediction_timestamps = []
        
        # Predecir para los próximos 6 intervalos (1 hora)
        for i in range(6):
            # Hacer predicción para el próximo paso
            next_pred = self.model.predict(last_sequence.reshape(1, self.time_steps, 4))[0]
            
            # Agregar predicción a la lista
            predictions.append(next_pred)
            
            # Generar timestamp para la predicción
            next_timestamp = self.last_timestamp + pd.Timedelta(minutes=10*(i+1))
            prediction_timestamps.append(next_timestamp)
            
            # Actualizar la secuencia deslizante
            last_sequence = np.roll(last_sequence, -1, axis=0)
            last_sequence[-1] = next_pred
        
        # Revertir la normalización
        predictions = self.scaler.inverse_transform(np.array(predictions))
        
        # Crear DataFrame con predicciones
        pred_df = pd.DataFrame(predictions, columns=self.features)
        pred_df['Timestamp'] = prediction_timestamps
        
        return pred_df
    
    def visualize_predictions(self, predictions):
        plt.figure(figsize=(15,10))
        for i, feature in enumerate(self.features):
            plt.subplot(2,2,i+1)
            plt.plot(predictions['Timestamp'], predictions[feature], marker='o')
            plt.title(f'Predicción de {feature}')
            plt.xticks(rotation=45)
            plt.xlabel('Timestamp')
            plt.ylabel(feature)
        plt.tight_layout()
        plt.show()
    
    def compare_last_data(self, predictions):
        # Mostrar los últimos datos reales
        print("\nÚltimos datos reales:")
        print(self.grouped_data.tail())
        
        print("\nPredicciones:")
        print(predictions)
        
        # Calcular diferencias porcentuales
        last_real = self.grouped_data[self.features].iloc[-1]
        first_pred = predictions[self.features].iloc[0]
        
        print("\nComparación y variación:")
        for feature in self.features:
            diff = first_pred[feature] - last_real[feature]
            pct_change = (diff / last_real[feature]) * 100
            print(f"{feature}: Último Real = {last_real[feature]:.2f}, "
                  f"Primera Predicción = {first_pred[feature]:.2f}, "
                  f"Variación = {pct_change:.2f}%")

In [3]:
# Flujo principal
def main():
    # Ruta en Google Drive (MODIFICAR SEGÚN TU ARCHIVO)
    file_path = 'datos3 - copia.txt'
    
    # Crear predictor
    predictor = MarketPredictor(file_path)
    
    # Entrenar modelo
    history = predictor.train_model()
    
    # Predecir próximo día (próxima hora en intervalos de 10 min)
    predicciones = predictor.predict_next_day()
    
    # Comparar con últimos datos
    predictor.compare_last_data(predicciones)
    
    # Visualizar predicciones
    predictor.visualize_predictions(predicciones)
    
    return predicciones

In [5]:
# Ejecutar script
predicciones = main()

C:\Users\pc\AppData\Local\Temp\ipykernel_12780\2189079508.py:13: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['10min_group'] = df['Timestamp'].dt.floor('10T')


Epoch 1/50
1497/1497 [==============================] - 13s 7ms/step - loss: 0.0041 - val_loss: 3.0063e-04
Epoch 2/50
 247/1497 [===>..........................] - ETA: 7s - loss: 7.4859e-04

KeyboardInterrupt: 

In [1]:
import tensorflow as tf

print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))
print("Dispositivos GPU detectados:", tf.config.list_physical_devices('GPU'))


GPUs disponibles: 0
Dispositivos GPU detectados: []
